
# Sensor Lab 1

We're going to be playing with sensor data today with the help of an Arduino Uno (or similar). By the end of this session, you should be able to interact with and plot data from various sensors using a combination of Python and C. This notebook should give you a starting point to work from.

*Note to anyone looking at this repository in December 2018: This is very much a work in progress.*

Prerequisites:
- Install pyserial (https://pypi.org/project/pyserial)

## 1.0 Load the relevant libraries


In [2]:
import serial # used to establish a serial connection to your Arduino 
import subprocess # used to communicate with and upload sketches to your Arduino via commandline
import time # used for various time functions

## 2.0 Load the Arduino code

This set of cells uploads the correct Arduino sketch to the Arduino, so you can start collecting the appropriate set of data.

You don't have to use is section of cells at all -- you can simply upload your code via the graphical Arduino IDE if you want. However, I like putting everything I can in one notebook.

The documentation on Arduino's command line interface can be found at https://github.com/arduino/Arduino/blob/master/build/shared/manpage.adoc


### 2.1 Specify the location of the arduino program script

In [3]:
#path name should be set to the location of your arduino program
arduinoProg = "~/arduino-1.8.8-linux64/arduino-1.8.8-linux64/arduino-1.8.8/arduino" 


### 2.2: Specify the desired sketch (assumes specific comment header structure)

In the cells below, we're reading in the first several (carefully crafted) comment lines from the Arduino script and using them to create a command line call to the arduino. 

Instructions: Comment out all but the arduinoScript line of interest in the cell immediately below this one, and execute the code cells in order.

In [13]:
# USER-MODIFIED CELL. Choose the correct arduino script for your project by uncommenting only the 
# appropriate arduinoScript line. Please note: for whatever reason you'll need the complete path name here

# sketch_pystart : this is the shell code for arduino - can be adapted to any project
arduinoScript = "/home/liz/cecs8001_projects/sensors_3b1/sketch_pystart/sketch_pystart.ino" 

# sketch_LEDdimmer : project 12 in Arduino beginner box; used to test i/o via serial
#arduinoScript = "/home/liz/cecs8001_projects/sensors_3b1/sketch_LEDdimmer/sketch_LEDdimmer.ino"

In [21]:
# This cell should work for all scripts above

scriptFile = open(arduinoScript,'r')
startLine = scriptFile.readline()[3:].strip()
actionLine = scriptFile.readline()[3:].strip()
boardLine = scriptFile.readline()[3:].strip()
portLine = scriptFile.readline()[3:].strip()
endLine = scriptFile.readline()[3:].strip()
scriptFile.close()

#I use the naming and commenting convention I've created to customize the error check for each script
scriptList = arduinoScript.split('/')
scriptName = scriptList[len(scriptList)-1]
scriptTitle = scriptName.split('.')[0]
scriptTitleStart = scriptTitle + "-begin"
scriptTitleEnd = scriptTitle + "-end"

# Minimalist error checking for sketch_pystart:
if (startLine != scriptTitleStart or endLine != scriptTitleEnd):
    print("Wrong file selected. Are your arduinoScript path and file name correct?");
else: 
    print ("The specified Arduino sketch has passed our minimal error checking protocol. ")

The specified Arduino sketch has passed our minimal error checking protocol. 


In the cell below, we create the string containing the command we wish to issue to the Arduino IDE. This command is equivalent to what you would do in the Arduino IDE when you upload the specified sketch to your Arduino board.

In [22]:
arduinoComm = arduinoProg + " --" + actionLine + " --board " + boardLine + " --port " + portLine + " " + arduinoScript
arduinoComm = arduinoComm + " -v -v -v -v" #verbose mode
print(arduinoComm)

~/arduino-1.8.8-linux64/arduino-1.8.8-linux64/arduino-1.8.8/arduino --upload --board arduino:avr:uno --port /dev/ttyACM0 /home/liz/cecs8001_projects/sensors_3b1/sketch_pystart/sketch_pystart.ino -v -v -v -v


### 2.3 Load your sketch

Below, we use the subprocess.call routine to pass the arduinoComm string to the command line, basically as if we were typing it directly into the bash terminal. 

**Notes:** 
- Since you're passing the command to the terminal, the complete result of that command appears in the terminal. Have a look. The verbose mode gives you more extensive output, which is useful for debugging. 
- This can take a little while. You're actually launching a java program to compile the sketch, upload it, etc, in the background.
- **IMPORTANT: If you have uploaded a different sketch previously, press the reset button on the Arduino board before executing the code below.**

In [23]:
errorCheck = subprocess.call(arduinoComm,shell=True)
print(errorCheck)

if errorCheck != 0:
    print("\n Arduino sketch upload failed! Result code = %s ... " %(errorCheck))
    if errorCheck == 1:
        print(" Build / upload failed.")
    if errorCheck == 2:
        print(" Sketch not found.")
    if errorCheck == 3:
        print(" Invalid command line input.")
    if errorCheck == 4:
        print(" Preference passed to --get-pref doesn't exist.")
else:
    print("\n Arduino sketch uploaded successfully!")

1

 Arduino sketch upload failed! Result code = 1 ... 
 Build / upload failed.


#### Troubleshooting: 

If you end up with anything other than success upon executing the cell above, here are some things to check:

- *Can you upload the script directly from the Arduino IDE?* If not, you may have run into some trouble caused by our use of a VirtualBox o/s rather than our host o/s; I'm still not sure why this happens. The likely error is given below. If you see this, try shutting down your virtual machine and repeating the exercise from an Arduino IDE run on your host machine. If this works, then upload an empty sketch to your arduino, restart your virtual machine, re-open the notebook, and try again. If this doesn't work, check all your connections and try again. If this still doesn't work, swap your Arduino for another board and try again.

>avrdude: stk500_recv(): programmer is not responding
>
>avrdude: stk500_getsync() attempt 1 of 10: not in sync: resp=0x00  

- *Is there an error in your arduinoComm string?* If you get an invalid command line message, this might be the problem. Check the printed output, correct any errors, and try again. 
- *Did you forget to press the reset button on your board?* The best sign of this the appearance of the following errors in the terminal output: 

>    avrdude: stk500_initialize(): (b) protocol error, expect=0x10, resp=0x04
>
>    avrdude: initialization failed, rc=-1
>
>         Double check connections and try again, or use -F to override
>         this check.


## 3.0 Open up a serial connection to your Arduino

In the next line, the first argument of serial.Serial is specific to your machine. If you are using the ubuntu VirtualBox distribution, this *should* work for you if you've properly set up your virtual machine serial port configuration within VBox. You can check which serial ports you have by typing 

   python -m serial.tools.list_ports
   
in a terminal.

_Note to self: Provide instructions for this_

The second argument of serial.Serial specifies the baud rate. This is the rate at which the device you have connected should communicate over the serial port. This is something you have (or will) specify in your Arduino code for this lab.

In [7]:
ser = serial.Serial('/dev/ttyS0',9600) 
print(ser.name) #checks that the name of the serial device has been defined properly

/dev/ttyS0


In [8]:
ser.flushInput()


### 3.1 Test the data link

In [12]:
time.sleep(2)
print(ser.in_waiting)
print(ser.out_waiting)
    
#while True:
#   byte = ser.read(size=1)
#  print(byte)

0
0
